In [18]:
 pip install datasets

In [19]:
pip install pytesseract

In [48]:
import os
from datasets import load_dataset

os.environ["HF_DATASETS_CACHE"] = "/content/hf_datasets"

data = load_dataset("dvgodoy/rvl_cdip_mini", split="train")

root_dir = "docs_data"
os.makedirs(root_dir, exist_ok=True)

for i, example in enumerate(data):
    label = str(example['category'])
    img = example['image']

    l=[int(1),int(2),int(3),int(4)]
    if example['label'] in l:
      label_dir = os.path.join(root_dir, label)
      os.makedirs(label_dir, exist_ok=True)

      img.save(os.path.join(label_dir, f"img_{i}.png"))


In [21]:

dataset_path = "/content/docs_data"
labels = [label for label in os.listdir(dataset_path)]
idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}
label2idx

{'handwritten': 0, 'advertisement': 1, 'form': 2, 'email': 3}

In [22]:
import pandas as pd
images = []
labels = []

for label_folder, _, file_names in os.walk(dataset_path):
  if label_folder != dataset_path:
    label = label_folder[19:]
    for _, _, image_names in os.walk(label_folder):
      relative_image_names = []
      for image in image_names:
        relative_image_names.append(dataset_path + "/" + label + "/" + image)
      images.extend(relative_image_names)
      labels.extend([label] * len (relative_image_names))

data = pd.DataFrame.from_dict({'image_path': images, 'label': labels})
data.tail()

,image_path,label
836,/content/docs_data/email/img_553.png,email
837,/content/docs_data/email/img_547.png,email
838,/content/docs_data/email/img_616.png,email
839,/content/docs_data/email/img_619.png,email
840,/content/docs_data/email/img_513.png,email


In [23]:
from datasets import Dataset
import pytesseract
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def normalize_box(box, width, height):
     return [
         int(1000 * (box[0] / width)),
         int(1000 * (box[1] / height)),
         int(1000 * (box[2] / width)),
         int(1000 * (box[3] / height)),
     ]

def apply_ocr(example):
        # get the image
        image = Image.open(example['image_path'])

        width, height = image.size

        # apply ocr to the image
        ocr_df = pytesseract.image_to_data(image, output_type='data.frame')
        float_cols = ocr_df.select_dtypes('float').columns
        ocr_df = ocr_df.dropna().reset_index(drop=True)
        ocr_df[float_cols] = ocr_df[float_cols].round(0).astype(int)
        ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
        ocr_df = ocr_df.dropna().reset_index(drop=True)

        # get the words and actual (unnormalized) bounding boxes
        #words = [word for word in ocr_df.text if str(word) != 'nan'])
        words = list(ocr_df.text)
        words = [str(w) for w in words]
        coordinates = ocr_df[['left', 'top', 'width', 'height']]
        actual_boxes = []
        for idx, row in coordinates.iterrows():
            x, y, w, h = tuple(row) # the row comes in (left, top, width, height) format
            actual_box = [x, y, x+w, y+h] # we turn it into (left, top, left+width, top+height) to get the actual box
            actual_boxes.append(actual_box)

        # normalize the bounding boxes
        boxes = []
        for box in actual_boxes:
            boxes.append(normalize_box(box, width, height))

        # add as extra columns
        assert len(words) == len(boxes)
        example['words'] = words
        example['bbox'] = boxes
        return example

dataset = Dataset.from_pandas(data)
updated_dataset = dataset.map(apply_ocr)

Map:   0%|          | 0/841 [00:00<?, ? examples/s]

/tmp/ipython-input-23-1052913753.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
/tmp/ipython-input-23-1052913753.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
/tmp/ipython-input-23-1052913753.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior

In [24]:
pip install transformers

In [26]:
from transformers import LayoutLMTokenizer
import torch

tokenizer = LayoutLMTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")

def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):
  words = example['words']
  normalized_word_boxes = example['bbox']

  assert len(words) == len(normalized_word_boxes)

  token_boxes = []
  for word, box in zip(words, normalized_word_boxes):
      word_tokens = tokenizer.tokenize(word)
      token_boxes.extend([box] * len(word_tokens))

  # Truncation of token_boxes
  special_tokens_count = 2
  if len(token_boxes) > max_seq_length - special_tokens_count:
      token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

  # add bounding boxes of cls + sep tokens
  token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

  encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
  # Padding of token_boxes up the bounding boxes to the sequence length.
  input_ids = tokenizer(' '.join(words), truncation=True)["input_ids"]
  padding_length = max_seq_length - len(input_ids)
  token_boxes += [pad_token_box] * padding_length
  encoding['bbox'] = token_boxes
  try:
    encoding['label'] = label2idx[example['label']]
  except:
    pass

  assert len(encoding['input_ids']) == max_seq_length
  assert len(encoding['attention_mask']) == max_seq_length
  assert len(encoding['token_type_ids']) == max_seq_length
  assert len(encoding['bbox']) == max_seq_length

  return encoding

In [27]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D
# we need to define the features ourselves as the bbox of LayoutLM are an extra feature
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'label': ClassLabel(names=['advertisement','email','form','handwritten']),
    'image_path': Value(dtype='string'),
    'words': Sequence(feature=Value(dtype='string')),
})

encoded_dataset = updated_dataset.map(lambda example: encode_example(example),
                                      features=features)

Map:   0%|          | 0/841 [00:00<?, ? examples/s]

In [28]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'bbox', 'attention_mask', 'token_type_ids', 'label'])
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=1, shuffle=True)
batch = next(iter(dataloader))

In [29]:
from transformers import LayoutLMForSequenceClassification
import torch
from transformers import logging

logging.set_verbosity_warning()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LayoutLMForSequenceClassification.from_pretrained("microsoft/layoutlm-base-uncased", num_labels=len(label2idx))
model.to(device)

Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMForSequenceClassification(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True

In [30]:
from torch.optim import AdamW
import torch

optimizer = AdamW(model.parameters(), lr=4e-5)

global_step = 0
num_train_epochs = 15
t_total = len(dataloader) * num_train_epochs # total number of training steps

#put the model in training mode
model.train()
for epoch in range(num_train_epochs):
  print("Epoch:", epoch)
  running_loss = 0.0
  correct = 0
  for batch in dataloader:
      input_ids = batch["input_ids"].to(device)
      bbox = batch["bbox"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      token_type_ids = batch["token_type_ids"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                      labels=labels)
      loss = outputs.loss

      running_loss += loss.item()
      predictions = outputs.logits.argmax(-1)
      correct += (predictions == labels).float().sum()

      # backward pass to get the gradients
      loss.backward()

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1

  print("Loss:", running_loss / batch["input_ids"].shape[0])
  accuracy = 100 * correct / len(data)
  print("Training accuracy:", accuracy.item())

Epoch: 0
Loss: 546.3634277340025
Training accuracy: 74.67300415039062
Epoch: 1
Loss: 367.0928169279359
Training accuracy: 83.94767761230469
Epoch: 2
Loss: 347.3707907893695
Training accuracy: 85.01783752441406
Epoch: 3
Loss: 345.42565913149156
Training accuracy: 85.61236572265625
Epoch: 4
Loss: 286.2122358828783
Training accuracy: 89.06063842773438
Epoch: 5
Loss: 172.72397994063795
Training accuracy: 93.6979751586914
Epoch: 6
Loss: 268.70978035754524
Training accuracy: 88.94173431396484
Epoch: 7
Loss: 184.31126820365898
Training accuracy: 92.50891876220703
Epoch: 8
Loss: 412.3436578293331
Training accuracy: 81.92627716064453
Epoch: 9
Loss: 263.27977240271866
Training accuracy: 90.963134765625
Epoch: 10
Loss: 165.44357100175694
Training accuracy: 92.50891876220703
Epoch: 11
Loss: 140.37530874891672
Training accuracy: 94.64922332763672
Epoch: 12
Loss: 198.18201660056366
Training accuracy: 90.6064224243164
Epoch: 13
Loss: 217.66377761773765
Training accuracy: 92.1521987915039
Epoch: 14
Lo

In [33]:
#test_img = Image.open("/content/form_test.webp")
test_data = pd.DataFrame.from_dict({'image_path': ['/content/form_test.webp']})
test_dataset = Dataset.from_pandas(test_data)
updated_test_dataset = test_dataset.map(apply_ocr)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [34]:
encoded_test_dataset = updated_test_dataset.map(lambda example: encode_example(example),
                                      features=features)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [35]:
type(encoded_test_dataset)

datasets.arrow_dataset.Dataset

In [41]:
input_ids      = torch.tensor(encoded_test_dataset["input_ids"]).to(device)
attention_mask = torch.tensor(encoded_test_dataset["attention_mask"]).to(device)
token_type_ids = torch.tensor(encoded_test_dataset["token_type_ids"]).to(device)
bbox           = torch.tensor(encoded_test_dataset['bbox']).to(device)

In [42]:
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask,
                token_type_ids=token_type_ids)


In [47]:
classification_logits = outputs.logits
classification_results = torch.softmax(classification_logits, dim=1).tolist()[0]

classes = ['advertisement','email','form','handwritten']
max_prob = np.argmax(classification_results)
print(classes[max_prob])

form
